In [1]:
#imports here
import ccxt


In [4]:
print(ccxt.exchanges.index("kraken"))

74


In [7]:
api_key = "cFT6E0pHVjn6TWTCA3QHi19zmT3LYXrRxKTmiYPBHqKBQfc8JZLX00kB"
api_secret = "7IEhBkHo+u2yH6oU2Uzd7BTJlT5eD/o0WxXRUISxzpQMv1W5RKsCG1COWMtUVT9GK5q8tMYAyYszpH3NPkm+1Q=="

In [42]:
api_key = "BEvXskqd7KKy+cT4LYoxePoD1oEyHolI0U4gH1l4U/UTwd0GsakSdQO8"
api_secret = "BxDwwiHKBj5zR+czwgsr1N/QBxGsaym6jECj3WyVbazPkpnQuM1/epsU7d6EJPMIvAou51ug0t3Cgsw5Qon7nA=="

In [43]:
exchange_class = getattr(ccxt,"kraken")

In [44]:
ex  = exchange_class({
    'apiKey':api_key,
    'secret':api_secret
})

In [41]:
address = ex.fetchDepositAddress(code="BNB",params = {'method' : 'Polygon (MATIC)'})
address

ExchangeError: Does not have currency code BNB

In [45]:
markets = ex.load_markets()

In [37]:
import time
import os
import requests
import urllib.parse
import hashlib
import hmac
import base64

def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

# Read Kraken API key and secret stored in environment variables
api_url = "https://api.kraken.com"


# Attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

# Construct the request and print the result
resp = kraken_request('/0/private/DepositMethods', {
    "nonce": str(int(1000*time.time())),
    "asset": "MATIC"
}, api_key, api_secret)

print(resp.json())

{'error': [], 'result': [{'method': 'Polygon (MATIC)', 'limit': False, 'address-setup-fee': '10.00000000', 'gen-address': True}]}


In [38]:
balance = ex.fetch_balance()
balance

{'info': {'error': [], 'result': {'UST': '26.81480500'}},
 'timestamp': None,
 'datetime': None,
 'UST': {'free': None, 'used': None, 'total': 26.814805},
 'free': {'UST': None},
 'used': {'UST': None},
 'total': {'UST': 26.814805}}

In [57]:
get_price("GBP"+"/USD",ex)

1.25211

In [47]:
# get price 
def get_price(pair,ex):
    orderbook = ex.fetch_order_book(pair)
    bid = orderbook['bids'][0][0] if len (orderbook['bids']) > 0 else None
    ask = orderbook['asks'][0][0] if len (orderbook['asks']) > 0 else None
    return ask

# get perc amount
def get_perc_amount(amount,perc):
    return round(amount  * perc , 8)

# script to watch usdt and convert to ust
def convert_to_ust():
    balance = ex.fetch_balance()
    markets = ex.load_markets()
    ust_min  = markets['UST/USDT']['limits']['amount']['min']
    total_usdt = balance['total']['USDT']
    trade_amount = get_perc_amount(total_usdt  / get_price("UST/USDT",ex) , 0.99)
    formated = ex.amount_to_precision("UST/USDT",trade_amount)
    if float(trade_amount) > float(ust_min):
        print("can be traded")
        res = ex.create_market_buy_order('UST/USDT',trade_amount)
        print(res)
    else:
        print("minimum not reached")

fiat_pairs = ["USDT/USD","USDT/GBP","USDT/EUR"]


In [44]:
def get_base_fiat_amount(pair,amount,ex):
    price = get_price(pair,ex)
    return round((amount / price),8)
def watch_convert_fiat():
    base = "USDT"
    fiats = ["USD","GBP","EUR"]
    markets = ex.load_markets()
    balance= ex.fetch_balance()
    for fiat in fiats:
        min_amount = markets[base+"/"+fiat]['limits']['amount']['min']
        fiat_balance = balance['total'].get(fiat,False)
        if fiat_balance:
            trade_amount = get_perc_amount(fiat_balance  / get_price(base+"/"+fiat,ex) , 0.99)
            formated =  ex.amount_to_precision(base+"/"+fiat,trade_amount)
            if float(trade_amount) > float(min_amount):
                print("can be traded")
                res = ex.create_market_buy_order(base+"/"+fiat,formated)
                print(res)
                trade_id = res.get("id",False)
                if trade_id:
                    print("success Fiat Trade")
                else:
                    print("failed Fiat Trade")
            else:
                print("minimum not reached for "+ fiat)

        else:
            print("thee is no balance for " + fiat)
watch_convert_fiat()
        
    
    

thee is no balance for USD
minimum not reached for GBP
thee is no balance for EUR


In [31]:
10 /get_price("USDT/GBP",ex)

13.024225058609012

In [22]:
symbols = ex.symbols
filtered_symbols = []
for symbol in symbols:
    if 'USD' in symbol:
        filtered_symbols.append(symbol)
        
print(filtered_symbols)

['1INCH/USD', 'AAVE/USD', 'ACA/USD', 'ADA/USD', 'ADA/USDT', 'AIR/USD', 'AKT/USD', 'ALCX/USD', 'ALGO/USD', 'ALICE/USD', 'ANKR/USD', 'ANT/USD', 'APE/USD', 'ASTR/USD', 'ATLAS/USD', 'ATOM/USD', 'AUD/USD', 'AUDIO/USD', 'AVAX/USD', 'AXS/USD', 'BADGER/USD', 'BAL/USD', 'BAND/USD', 'BAT/USD', 'BCH/USD', 'BCH/USDT', 'BICO/USD', 'BNC/USD', 'BNT/USD', 'BOND/USD', 'BTC/USD', 'BTC/USDC', 'BTC/USDT', 'CHZ/USD', 'COMP/USD', 'CQT/USD', 'CRV/USD', 'CTSI/USD', 'CVX/USD', 'DAI/USD', 'DAI/USDT', 'DASH/USD', 'DOGE/USD', 'DOGE/USDT', 'DOT/USD', 'DOT/USDT', 'DYDX/USD', 'ENJ/USD', 'ENS/USD', 'EOS/USD', 'EOS/USDT', 'ETC/USD', 'ETH/USD', 'ETH/USDC', 'ETH/USDT', 'EUR/USD', 'EWT/USD', 'FIDA/USD', 'FIL/USD', 'FLOW/USD', 'FXS/USD', 'GALA/USD', 'GBP/USD', 'GHST/USD', 'GLMR/USD', 'GNO/USD', 'GRT/USD', 'ICP/USD', 'ICX/USD', 'IMX/USD', 'INJ/USD', 'JASMY/USD', 'KAR/USD', 'KAVA/USD', 'KEEP/USD', 'KILT/USD', 'KIN/USD', 'KINT/USD', 'KNC/USD', 'KP3R/USD', 'KSM/USD', 'LINK/USD', 'LINK/USDT', 'LPT/USD', 'LRC/USD', 'LSK/USD', '

In [23]:
def get_price(pair,ex):
    orderbook = ex.fetch_order_book(pair)
    bid = orderbook['bids'][0][0] if len (orderbook['bids']) > 0 else None
    ask = orderbook['asks'][0][0] if len (orderbook['asks']) > 0 else None
    return ask



In [24]:
price = get_price("UST/USDT",ex)

In [25]:
balance['total']['USDT']

28.90000001

In [26]:
def get_perc_amount(amount,perc):
    return round(amount  * perc , 8)


In [29]:
trade_amount = get_perc_amount(balance['total']['USDT']  * price,0.95)
print(trade_amount)

27.46872751


In [32]:
res = ex.create_market_buy_order('UST/USDT',trade_amount)
print(res)

{'id': 'OAVOTI-2IEEV-J6K4IZ', 'clientOrderId': None, 'info': {'txid': ['OAVOTI-2IEEV-J6K4IZ'], 'descr': {'order': 'buy 27.46872751 USTUSDT @ market'}}, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'status': None, 'symbol': 'UST/USDT', 'type': 'market', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopPrice': None, 'cost': None, 'amount': 27.46872751, 'filled': None, 'average': None, 'remaining': None, 'fee': None, 'trades': None, 'fees': []}


In [31]:
ex.has['createMarketOrder']

True

In [38]:
total_usdt = balance['total']['USDT']
trade_amount = get_perc_amount(total_usdt  * get_price("UST/USDT",ex) , 0.99)
formated = ex.amount_to_precision("UST/USDT",trade_amount)

In [39]:
formated

'1.34457213'

In [58]:
markets = ex.markets
markets['UST/USDT']


{'percentage': True,
 'tierBased': True,
 'taker': 0.002,
 'maker': 0.002,
 'tiers': {'taker': [[0, 0.0026],
   [50000, 0.0024],
   [100000, 0.0022],
   [250000, 0.002],
   [500000, 0.0018],
   [1000000, 0.0016],
   [2500000, 0.0014],
   [5000000, 0.0012],
   [10000000, 0.0001]],
  'maker': [[0, 0.0016],
   [50000, 0.0014],
   [100000, 0.0012],
   [250000, 0.001],
   [500000, 0.0008],
   [1000000, 0.0006],
   [2500000, 0.0004],
   [5000000, 0.0002],
   [10000000, 0.0]]},
 'precision': {'amount': 8, 'price': 4},
 'limits': {'amount': {'min': 5.0, 'max': 100000000.0},
  'price': {'min': 0.0001, 'max': None},
  'cost': {'min': 0, 'max': None},
  'leverage': {'max': 1}},
 'id': 'USTUSDT',
 'symbol': 'UST/USDT',
 'base': 'UST',
 'quote': 'USDT',
 'baseId': 'UST',
 'quoteId': 'USDT',
 'darkpool': False,
 'info': {'altname': 'USTUSDT',
  'wsname': 'UST/USDT',
  'aclass_base': 'currency',
  'base': 'UST',
  'aclass_quote': 'currency',
  'quote': 'USDT',
  'lot': 'unit',
  'pair_decimals': '4',

In [46]:
markets['UST/USDT']['limits']['amount']['min']


5.0

In [14]:
res = ex.create_market_sell_order('UST/USDT',balance['total']['UST'])
res

{'id': 'OE2YTK-C5RSS-V2XZIJ',
 'clientOrderId': None,
 'info': {'txid': ['OE2YTK-C5RSS-V2XZIJ'],
  'descr': {'order': 'sell 55.45301942 USTUSDT @ market'}},
 'timestamp': None,
 'datetime': None,
 'lastTradeTimestamp': None,
 'status': None,
 'symbol': 'UST/USDT',
 'type': 'market',
 'timeInForce': None,
 'postOnly': None,
 'side': 'sell',
 'price': None,
 'stopPrice': None,
 'cost': None,
 'amount': 55.45301942,
 'filled': None,
 'average': None,
 'remaining': None,
 'fee': None,
 'trades': [],
 'fees': []}

In [16]:
res = ex.create_market_sell_order('USDT/GBP',balance['total']['USDT'])
res

{'id': 'OZRTR5-ZNP4X-SSAA5J',
 'clientOrderId': None,
 'info': {'txid': ['OZRTR5-ZNP4X-SSAA5J'],
  'descr': {'order': 'sell 55.81388077 USDTGBP @ market'}},
 'timestamp': None,
 'datetime': None,
 'lastTradeTimestamp': None,
 'status': None,
 'symbol': 'USDT/GBP',
 'type': 'market',
 'timeInForce': None,
 'postOnly': None,
 'side': 'sell',
 'price': None,
 'stopPrice': None,
 'cost': None,
 'amount': 55.81388077,
 'filled': None,
 'average': None,
 'remaining': None,
 'fee': None,
 'trades': [],
 'fees': []}

NameError: name 'get_price' is not defined

In [8]:
add = "terra1f2fgl9wyuz8k2qdj2ywtk3ksz7qgxvdcs2y7xd"
tags = None
params = {
    'key': 'xo ust',
}
w = ex.withdraw("UST","2",add,tags,params)
print(w)

{'info': {'refid': 'BTONFTP-VR23ME-UDNKIJ'}, 'id': 'BTONFTP-VR23ME-UDNKIJ'}


In [11]:
withdrawal = ex.fetchWithdrawals(code="UST")

In [12]:
withdrawal

[{'info': {'type': 'withdrawal',
   'method': 'TerraUSD',
   'aclass': 'currency',
   'asset': 'UST',
   'refid': 'BTONFTP-VR23ME-UDNKIJ',
   'txid': 'c6553cf65c7be33fc213fd6be9ab89ff4fd6139652bb783073a3a4e2ed30d4ad',
   'info': 'terra130v6rqy3pecgcja5t4juhlulfq8vmxjwg7vswm',
   'amount': '1.000000',
   'fee': '1.000000',
   'time': '1649424778',
   'status': 'Success'},
  'id': 'BTONFTP-VR23ME-UDNKIJ',
  'currency': 'UST',
  'amount': 1.0,
  'network': None,
  'address': 'terra130v6rqy3pecgcja5t4juhlulfq8vmxjwg7vswm',
  'addressTo': None,
  'addressFrom': None,
  'tag': None,
  'tagTo': None,
  'tagFrom': None,
  'status': 'ok',
  'type': 'withdrawal',
  'updated': None,
  'txid': 'c6553cf65c7be33fc213fd6be9ab89ff4fd6139652bb783073a3a4e2ed30d4ad',
  'timestamp': 1649424778000,
  'datetime': '2022-04-08T13:32:58.000Z',
  'fee': {'currency': 'UST', 'cost': 1.0}},
 {'info': {'type': 'withdrawal',
   'method': 'TerraUSD',
   'aclass': 'currency',
   'asset': 'UST',
   'refid': 'BTOXTX5-FS

In [82]:
import requests
def handleResp(resp):
    if (resp['status']):
        return resp['message']
    else:
        return False
api_url = "http://localhost:3010/"
resp =  requests.post(api_url + "api/anchor")
print(handleResp(resp.json()))

False


In [81]:
resp2 = requests.post(api_url + "api/anchor/deposit")
print(handleResp(resp2.json()))

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))

In [75]:
resp.json()

{'status': True, 'message': 'ready to deposit'}